In [24]:
!pip install tensorflow --quiet
!pip install pandas --quiet
!pip install scikit-learn --quiet


In [47]:
from google.colab import files
uploaded_csv=files.upload()

Saving tatasteel.csv to tatasteel (4).csv
Saving tcs.csv to tcs (4).csv
Saving tsla.csv to tsla (4).csv


In [48]:
import pandas as pd

# Load CSVs
tcs = pd.read_csv('tcs.csv')
tatasteel = pd.read_csv('tatasteel.csv')
tsla = pd.read_csv('tsla.csv')

In [49]:
tcs['stock'] = 'TCS'
tatasteel['stock'] = 'TataSteel'
tsla['stock'] = 'TSLA'

In [50]:
combined_df = pd.concat([tcs, tatasteel, tsla], ignore_index=True)

In [51]:
combined_df = combined_df.sort_values(by=['stock','date'])
combined_df.reset_index(drop=True, inplace=True)


In [52]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
combined_df['close_scaled'] = scaler.fit_transform(combined_df['close'].values.reshape(-1,1))


In [53]:
import joblib
joblib.dump(scaler, 'combined_scaler.pkl')


['combined_scaler.pkl']

In [54]:
import numpy as np

sequence_length = 60
X, y = [], []

for stock in combined_df['stock'].unique():
    stock_data = combined_df[combined_df['stock']==stock]['close_scaled'].values
    for i in range(sequence_length, len(stock_data)):
        X.append(stock_data[i-sequence_length:i])
        y.append(stock_data[i])

X = np.array(X).reshape(-1, sequence_length, 1)
y = np.array(y)



In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.fit(X, y, epochs=10, batch_size=32)
model.save('combined_stock_model.h5')


Epoch 1/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 26s 61ms/step - loss: 0.0058 - mae: 0.0378
Epoch 2/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - loss: 9.7715e-04 - mae: 0.0193
Epoch 3/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 23s 62ms/step - loss: 6.1018e-04 - mae: 0.0151
Epoch 4/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 40s 61ms/step - loss: 6.5799e-04 - mae: 0.0155
Epoch 5/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - loss: 6.0653e-04 - mae: 0.0157
Epoch 6/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - loss: 5.5808e-04 - mae: 0.0152
Epoch 7/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 40s 57ms/step - loss: 7.7800e-04 - mae: 0.0186
Epoch 8/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 42s 60ms/step - loss: 5.0376e-04 - mae: 0.0148
Epoch 9/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - loss: 5.2643e-04 - mae: 0.0157
Epoch 10/10
366/366 ━━━━━━━━━━━━━━━━━━━━ 21s 57ms/step - loss: 5.0809e-04 - mae: 0.0152


In [58]:
stocks = ['TCS', 'TataSteel', 'TSLA']

for stock in stocks:
    last_60_prices = combined_df[combined_df['stock']==stock]['close'].values[-60:]
    last_60_scaled = scaler.transform(last_60_prices.reshape(-1,1)).reshape(1,60,1)

    # Add verbose=0 to suppress the progress bar
    predicted_scaled = model.predict(last_60_scaled, verbose=0)

    predicted_actual = scaler.inverse_transform(predicted_scaled.reshape(-1,1))
    print(f"{stock} next day predicted price: {predicted_actual[0][0]}")


TCS next day predicted price: 3422.33349609375
TataSteel next day predicted price: 1097.989501953125
TSLA next day predicted price: 712.2086791992188


In [59]:
from google.colab import files

# Download your trained model
files.download('combined_stock_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [60]:
# Download the scaler you saved
files.download('combined_scaler.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>